In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from urllib.parse import urlparse, parse_qs, parse_qsl
import time
from datetime import datetime, timedelta
import re
from selenium.common.exceptions import NoSuchElementException
import pandas as pd

In [4]:
from selenium.webdriver.chrome.service import Service as ChromeService

service = Service()
options = webdriver.ChromeOptions()
# options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# driver = webdriver.Chrome(service=service, options=options)
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [5]:
site_url = "https://www.teamblind.com/kr/topics/부동산"

In [ ]:
driver.get(site_url)
driver.implicitly_wait(3)
time.sleep(2)  # 페이지 로딩 대기

In [4]:
# Get Link on div
sub_link = driver.find_element(By.CLASS_NAME, "tit").find_element(By.XPATH, 'h3/a').get_attribute('href')
sub_link

'https://www.teamblind.com/kr/post/%EB%8B%A4%EB%A5%B8%EA%B1%B4-%EB%AA%B0%EB%9D%BC%EB%8F%84-%EB%8F%88-%EB%A7%8E%EC%9D%80-%EC%82%AC%EB%9E%8C%EC%9D%B4-%EB%A7%8E%EB%8B%A4%EB%8A%94-%EB%A7%90%EC%9D%80-%EA%B0%9C%EA%B5%AC%EB%9D%BC%EC%95%BC%E3%85%8E%E3%85%8E-26nihsck'

In [ ]:
# 게시글 진입
driver.get(sub_link)

In [ ]:
# 게시글 + 댓글 크롤링 start
# 제목 (<h2>)
driver.find_element(By.CLASS_NAME, "article-view-head").find_element(By.CSS_SELECTOR, "h2").text

'다른건 몰라도 돈 많은 사람이 많다는 말은 개구라야ㅎㅎ'

In [38]:
# 게시글
driver.find_element(By.CLASS_NAME, "article-view-contents").find_element(By.ID, "contentArea").text

'내가 담당했던 업무 (지금은 아님)가\n\nKb부자리포트 같은 곳에 반영되는 고객들 금융자산 현황 만들었었거덩\n\n당행뿐 아니라 타행들것도 같이 볼수있는데 거의 전국민의 금융자산 현황을 매일 보는 업무를 했었음ㅎㅎ 개인정보를 본건아니고 전체적인 현황 자료\n\n그거보면서 느낀게\n\n돈 많은 사람 겁나 많다는거 다 개구라 라는걸 느꼈다ㅎㅎ\n\n대부분이 빚더미 위에 올라서 돈많은 척하는거지\n\n솔직히 금융자산 자체가 비슷한 경제규모의 타국에 비해 현저히 적음\n\n지들끼리는 나 빼고 다 돈 많아~ 우리나라에 돈많은 사람들이 얼마나 많은데 이러고있는게 웃음벨\n\n통계에 금융자산 10억이 우리나라에 0.8%도 안되는게 레알인데 저기엔 채권이나 펀드 주식 등 모든 금융자산이 포함되어있어서 당장 현금화시킬수있는 돈 기준으로 하면 진짜 돈많은 사람은 거의 없다\n\n한마디로 집 빼면 대부분 빚더미에 앉아있는 위태위태한 사람들이 거의 다더라..'

In [ ]:
# 작성회사
driver.find_element(By.CLASS_NAME, "name").find_element(By.CLASS_NAME, "point").text

'SC제일은행'

In [25]:
# 작성ID
driver.find_element(By.CLASS_NAME, "name").text.split('\n')[-1]

'· l********  '

In [15]:
# 작성일(절대날짜 추출)
def convert_date(relative_text):
    """
    상대적인 시간 표현을 절대 날짜로 변환하는 함수
    
    Args:
        relative_text (str): '방금', '2분', '1시간', '1일', '1달' 등의 텍스트
    
    Returns:
        str: '%Y-%m-%d' 형태의 절대 날짜
    """
    # 현재 날짜 및 시간
    now = datetime.now()
    # 공백 제거 및 소문자 변환
    text = relative_text.strip()
    # '방금'인 경우 오늘 날짜 반환
    if text == '방금':
        return now.strftime('%Y-%m-%d')
    
    # 숫자와 단위 추출을 위한 정규표현식
    pattern = r'(\d+)(분|시간|일|달|주)'
    match = re.search(pattern, text)
    
    if not match:
        # 패턴이 매치되지 않으면 오늘 날짜 반환
        return now.strftime('%Y-%m-%d')
    
    number = int(match.group(1))
    unit = match.group(2)
    
    # 단위별로 날짜 계산
    if unit == '분':
        target_date = now - timedelta(minutes=number)
    elif unit == '시간':
        target_date = now - timedelta(hours=number)
    elif unit == '일':
        target_date = now - timedelta(days=number)
    elif unit == '주':
        target_date = now - timedelta(weeks=number)
    elif unit == '달':
        # 달의 경우 대략 30일로 계산 (정확하지 않을 수 있음)
        target_date = now - timedelta(days=number * 30)
    else:
        target_date = now
    
    return target_date.strftime('%Y-%m-%d')

In [32]:
# 작성일(현재시점 며칠 전)
write_date_before=driver.find_element(By.CLASS_NAME, "date").text.split('\n')[-1]
write_date_before

'4일'

In [ ]:
# 작성일(절대시간 기준)
convert_date(write_date_before)

'2025-06-28'

In [35]:
# 좋아요
driver.find_element(By.CLASS_NAME, "like").text.split('\n')[-1]

'201'

In [37]:
# 댓글수
driver.find_element(By.CLASS_NAME, "cmt").text.split('\n')[-1]

'145'

In [ ]:
# 댓글 세부내용 추출
# 댓글이 더 있는경우 모두 펼친다
# 이 기능은 로그인 되어있는 경우만 가능
driver.find_element(By.CLASS_NAME, "btn-reply").click()

In [ ]:
# Blind 인증 요구 시 다음 Element 가 생긴다
# 해당 Element가 존재한다면 로그인 불가로 간주하고 현재 보여지는 댓글만 가져오고 끝낸다
# 해당 Element가 없다면, try-catch 로 계속 진행하도록 수행(로그인 된 것으로 간주)
driver.find_element(By.XPATH, "/html/body/div/div/div/main/div[3]/div[1]/div")

<selenium.webdriver.remote.webelement.WebElement (session="8483d6fa5b242ce85251fb8bae404044", element="f.6FE7B10A4ABC9024921797A9D4DD568D.d.B505AED63D3E7AA3F6906F3382FF69E0.e.11677")>

In [79]:
comment_element_list=driver.find_elements(By.CLASS_NAME, "comment_area")
comment_element_list[2].text

'한국전력공사\n· i*********\n또또 주변도르\n작성일\n3일\n좋아요수\n14'

In [102]:
print(comment_element_list[7].text.split('·')[0]) # 리플 작성회사
print(comment_element_list[7].text.split('·')[1].split('\n')[0]) # 리플 작성ID
print(comment_element_list[7].text.split('·')[1].split(comment_element_list[7].text.split('·')[1].split('\n')[0])[1].split('작성일')[0]) # 리플 내용

한양대학교교직원

 i*******

와 중소기업연구원은
ㄹㅇ 멍청한게신념도 강해서
때려죽여도 남의말은 안듣네

너무추하다ㅋㅋ



In [16]:
def get_comment_txt(comment) :
    comment_map = {}
    comment_map['firm']=comment.text.split('·')[0] # 리플 작성회사
    comment_map['id']=comment.text.split('·')[1].split('\n')[0]  # 리플 작성ID
    # comment_map['contents']=comment.text.split('·')[1].split('\n')[1]  # 리플 내용
    comment_map['contents']=comment.text.split('·')[1].split(comment.text.split('·')[1].split('\n')[0])[1].split('작성일')[0] # 리플 내용
    
    return comment_map

In [119]:
comment_list = list(map(lambda x : get_comment_txt(x), comment_element_list))
comment_list

[{'firm': '변호사 ', 'id': ' P*****', 'contents': '\n30살 연봉 1억이면 상위 몇퍼야\n'},
 {'firm': '새회사 ',
  'id': ' 세********',
  'contents': '\n나 만 28인데 부채제외 주식 부동산 차량 다 현금화하면 7억은 됨 근데 내 주변에 친구들은 다 더 부자야\n생각보다 부자 많어 ㅠㅡㅠ\n'},
 {'firm': '한국전력공사\n', 'id': ' i*********', 'contents': '\n또또 주변도르\n'},
 {'firm': '새회사 ',
  'id': ' I********',
  'contents': '\n우리나라에 순자산 32억 이상 있는 가구는 20만이 넘어\n\n“상위 1%“인 20만 가구는 서울의 한 구 전체, 혹은 중소도시 하나를 통째로 구성할 정도로 엄청나게 큰 집단입니다.\n단순히 ‘극소수’가 아닌, 꽤 두터운 규모의 인원이라는 걸 알 수 있죠.\n\n참고로 금융자산 10억 이상은 46만명임\n46만이면 성남시 분당구 인구수랑 같음\n\n이렇게 많은 사람들이 5000만명 대비는 작으니까 우리나라 부자 ㅈ도없엌ㅋㅋㅋㅋ 라고할수 있을까?\n\n손바닥으로 태양을 가리고 어둡다고 하는거지\n\n그리고 부자들을 모여살아 끼리끼리\n\n본인주변에 부자가 없다면\n왜일까 생각해 보자\n'},
 {'firm': 'SC제일은행\n',
  'id': ' l********',
  'contents': '\n작성자\nㅋㅋㅋ 내가 그 순자산 32억이상 있는 가구중 한명인데. 나도 부동산에 묶여있어서 사실 현금이 별로없어. 그리고 현금화? 그게 말이쉽지 부동산 파는건 쉬운일이 아니다. 양도세도 그렇고\n\n대다수 32억이상 있는 가구들도 나와 사정이 별반 다르지않을거야 그러니 금융자산 있는 사람들이 우리나라엔 별로없는것임.\n'},
 {'firm': '신영증권\n',
  'id': ' 신****',
  'contents': '\nㅋㅋㅋ 은행이 자료를가지고 말해도\n"주변도르" 

In [ ]:
# 게시물 이동 후 모든 컨텐츠를 가져오는 함수 definition
def get_article(link) : 
    try :
        driver.get(link)
        article = {}
        # 제목 (<h2>)
        title=driver.find_element(By.CLASS_NAME, "article-view-head").find_element(By.CSS_SELECTOR, "h2").text
        # 게시글
        body=driver.find_element(By.CLASS_NAME, "article-view-contents").find_element(By.ID, "contentArea").text
        # 작성회사
        writer_firm=driver.find_element(By.CLASS_NAME, "name").find_element(By.CLASS_NAME, "point").text
        # 작성ID
        writer_id=driver.find_element(By.CLASS_NAME, "name").text.split('\n')[-1]

        # 작성일
        write_date_before=driver.find_element(By.CLASS_NAME, "date").text.split('\n')[-1]
        write_date=convert_date(write_date_before)

        # 좋아요
        like_cnt=driver.find_element(By.CLASS_NAME, "like").text.split('\n')[-1]

        # 댓글수
        comment_cnt=driver.find_element(By.CLASS_NAME, "cmt").text.split('\n')[-1]

        # 댓글 세부내용 추출
        # 댓글이 더 있는경우 모두 펼친다
        # 이 기능은 로그인 되어있는 경우만 가능
        while True :
            # Blind 인증 요구 시 다음 Element 가 생긴다
            # 해당 Element가 존재한다면 로그인 불가로 간주하고 현재 보여지는 댓글만 가져오고 끝낸다
            # 해당 Element가 없다면, try-catch 로 계속 진행하도록 수행(로그인 된 것으로 간주)
            try :
                try :
                    driver.find_element(By.CLASS_NAME, "btn-reply").click()
                except :
                    break
                
                if(driver.find_element(By.XPATH, "/html/body/div/div/div/main/div[3]/div[1]/div")) :
                    # driver.find_element(By.CLASS_NAME, "btn-cls").click()
                    driver.find_element(By.CSS_SELECTOR, "#wrap > div:nth-child(4) > div:nth-child(1) > div > div.v--modal-box.v--modal.v--size-s > div > div.ly-modal.ly-signin > div.top-area > button").click()
                    break
            except :
                continue
            
    
        comment_element_list=driver.find_elements(By.CLASS_NAME, "comment_area")
        comment_map = [get_comment_txt(x) for x in comment_element_list]
        
        # Make tuple
        article['title']=title
        article['body']=body
        article['firm']=writer_firm
        article['id']=writer_id
        article['like_cnt']=like_cnt
        article['comment_cnt']=comment_cnt
        article['comment_map']=comment_map
        
        print(f"[세부 글] {article}")
    
        driver.back()
    except :
        return None
 
    return article

In [141]:
get_article(sub_link)

{'title': '다른건 몰라도 돈 많은 사람이 많다는 말은 개구라야ㅎㅎ',
 'body': '내가 담당했던 업무 (지금은 아님)가\n\nKb부자리포트 같은 곳에 반영되는 고객들 금융자산 현황 만들었었거덩\n\n당행뿐 아니라 타행들것도 같이 볼수있는데 거의 전국민의 금융자산 현황을 매일 보는 업무를 했었음ㅎㅎ 개인정보를 본건아니고 전체적인 현황 자료\n\n그거보면서 느낀게\n\n돈 많은 사람 겁나 많다는거 다 개구라 라는걸 느꼈다ㅎㅎ\n\n대부분이 빚더미 위에 올라서 돈많은 척하는거지\n\n솔직히 금융자산 자체가 비슷한 경제규모의 타국에 비해 현저히 적음\n\n지들끼리는 나 빼고 다 돈 많아~ 우리나라에 돈많은 사람들이 얼마나 많은데 이러고있는게 웃음벨\n\n통계에 금융자산 10억이 우리나라에 0.8%도 안되는게 레알인데 저기엔 채권이나 펀드 주식 등 모든 금융자산이 포함되어있어서 당장 현금화시킬수있는 돈 기준으로 하면 진짜 돈많은 사람은 거의 없다\n\n한마디로 집 빼면 대부분 빚더미에 앉아있는 위태위태한 사람들이 거의 다더라..',
 'firm': 'SC제일은행',
 'id': '· l********  ',
 'like_cnt': '201',
 'comment_cnt': '145',
 'comment_map': [{'firm': '변호사 ',
   'id': ' P*****',
   'contents': '\n30살 연봉 1억이면 상위 몇퍼야\n'},
  {'firm': '새회사 ',
   'id': ' 세********',
   'contents': '\n나 만 28인데 부채제외 주식 부동산 차량 다 현금화하면 7억은 됨 근데 내 주변에 친구들은 다 더 부자야\n생각보다 부자 많어 ㅠㅡㅠ\n'},
  {'firm': '한국전력공사\n', 'id': ' i*********', 'contents': '\n또또 주변도르\n'},
  {'firm': '새회사 ',
   'id': ' I********',
   'contents': '\n우리나라에 순자산 32억

In [ ]:
# Get Pre-Text on div
driver.find_element(By.CLASS_NAME, "tit").find_element(By.XPATH, 'h3/a').text

'전세가 병신짓인걸 모르는 사람이 많다는게 놀랍다'

In [146]:
# 최초 설정
a=driver.find_element(By.CLASS_NAME, "article-list-pre")
a.text

'NOW\n다른건 몰라도 돈 많은 사람이 많다는 말은 개구라야ㅎㅎ\n내가 담당했던 업무 (지금은 아님)가 Kb부자리포트 같은 곳에 반영되는 고객들 금융자산 현황 만들었었거덩 당행뿐 아니라 타행들것도 같이 볼수있는데 거의 전국민의 금융자산 현황을 매일 보는 업무를 했었음ㅎㅎ 개인정보를 본건아니고 전체적인 현황 자료 그거보면서 느낀게 돈 많은 사람 겁나 많다는거 다 개구라 라는걸 느꼈다ㅎㅎ 대부분이 빚더미 위에 올라서 돈많은 척하는거지 솔직히 금융자산 자체가 비슷한 경제규모의 타국에 비해 현저히 적음 지들끼리는 나 빼고 다 돈 많아~ 우리나라에 돈많은 사람들이 얼마나 많은데 이러고있는\nSC제일은행 · l********\n조회수\n15K\n좋아요\n201\n댓글\n145\n작성시간\n4일\n북마크'

In [147]:
a.find_element(By.CLASS_NAME, "tit").find_element(By.XPATH, 'h3/a').text

'다른건 몰라도 돈 많은 사람이 많다는 말은 개구라야ㅎㅎ'

In [148]:
a.find_element(By.CLASS_NAME, "tit").find_element(By.XPATH, 'h3/a').get_attribute('href')

'https://www.teamblind.com/kr/post/%EB%8B%A4%EB%A5%B8%EA%B1%B4-%EB%AA%B0%EB%9D%BC%EB%8F%84-%EB%8F%88-%EB%A7%8E%EC%9D%80-%EC%82%AC%EB%9E%8C%EC%9D%B4-%EB%A7%8E%EB%8B%A4%EB%8A%94-%EB%A7%90%EC%9D%80-%EA%B0%9C%EA%B5%AC%EB%9D%BC%EC%95%BC%E3%85%8E%E3%85%8E-26nihsck'

In [149]:
a.find_element(By.CLASS_NAME, "tit").find_element(By.CLASS_NAME, 'pre-txt').text

'내가 담당했던 업무 (지금은 아님)가 Kb부자리포트 같은 곳에 반영되는 고객들 금융자산 현황 만들었었거덩 당행뿐 아니라 타행들것도 같이 볼수있는데 거의 전국민의 금융자산 현황을 매일 보는 업무를 했었음ㅎㅎ 개인정보를 본건아니고 전체적인 현황 자료 그거보면서 느낀게 돈 많은 사람 겁나 많다는거 다 개구라 라는걸 느꼈다ㅎㅎ 대부분이 빚더미 위에 올라서 돈많은 척하는거지 솔직히 금융자산 자체가 비슷한 경제규모의 타국에 비해 현저히 적음 지들끼리는 나 빼고 다 돈 많아~ 우리나라에 돈많은 사람들이 얼마나 많은데 이러고있는'

In [150]:
a.find_element(By.CLASS_NAME, "sub").find_element(By.CLASS_NAME, 'name').text

'SC제일은행 · l********'

In [260]:
# Get Next Sibling Object from div
a=a.find_element(By.XPATH, './following-sibling::div')
print(a.get_attribute('class'))
a.text

article-list-pre nth-even


'현금부자 많다는거 다 구라엿네 ;;\nhttps://n.news.naver.com/article/025/0003452258?sid=101 6억이상 받아서 가격 받혔던거라 조정 꽤 나올듯... 아니 현금부자들 많아서 다 대출 적게받았다면서 형들 이게 무슨일이야\n삼성바이오로직스 · i*********\n조회수\n947\n좋아요\n4\n댓글\n7\n작성시간\n6시간\n북마크'

In [159]:
a.get_attribute('class').endswith('ad')

True

In [254]:
# If None of next sibling from article-list, scrolling on the DOM body and repeat(recursive) to get sibling object
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(2)    

In [19]:
def get_article_list(url, crawl_cnt) :
    try :
        driver.get(url)
        driver.implicitly_wait(3)
        time.sleep(2)  # 페이지 로딩 대기

        article_pre_data = []

        # 게시물 첫번째거 설정(최초 1회, 이후에는 sibling 으로 설정)
        article_preview=driver.find_element(By.CLASS_NAME, "article-list-pre")

        for i in range(0, crawl_cnt+1) :
            article_object={}
            
            # 광고로 게시물인 경우 넘어간다
            # 게시물 text 가 'NOW\n' or 'HOT\n' 으로 시작하면 추천게시물이므로 넘어간다
            # 이외의 경우 반복 게시물 크롤링 시작
            if(article_preview.get_attribute('class').lower().endswith('ad') == False and
                article_preview.text.startswith('HOT\n') == False and
                article_preview.text.startswith('NOW\n') == False) :
                #title on list
                title = article_preview.find_element(By.CLASS_NAME, "tit").find_element(By.XPATH, 'h3/a').text
                
                # article link
                link = article_preview.find_element(By.CLASS_NAME, "tit").find_element(By.XPATH, 'h3/a').get_attribute('href')
                
                # article pre-text
                pre_text = article_preview.find_element(By.CLASS_NAME, "tit").find_element(By.CLASS_NAME, 'pre-txt').text
                
                # article firm
                firm = article_preview.find_element(By.CLASS_NAME, "sub").find_element(By.CLASS_NAME, 'name').text.split('·')[0]
                
                # article id
                id = article_preview.find_element(By.CLASS_NAME, "sub").find_element(By.CLASS_NAME, 'name').text.split('·')[1]
            
                # article date
                date_before = article_preview.find_element(By.CLASS_NAME, "sub").find_element(By.CLASS_NAME, 'info_fnc').find_element(By.CLASS_NAME, "past").text.split('작성시간\n')[1]
                date=convert_date(date_before)
            
                article_object['title']=title
                article_object['link']=link
                article_object['pre_text']=pre_text
                article_object['firm']=firm
                article_object['id']=id
                article_object['date']=date
                
                print(f"[{i}번째] {article_object}")
                
                article_pre_data.append(article_object)
                
            try :
                article_preview = article_preview.find_element(By.XPATH, './following-sibling::div')
            # 스크롤링의 끝인 경우 새로운 스크롤링 수행 후 계속 크로링 수행
            except NoSuchElementException as e:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)    
                
                print("### 스크롤링 완료###")
    finally :
        return article_pre_data

In [20]:
list = get_article_list(site_url, 50)

[3번째] {'title': '진짜 거래도 안해본 부린이들 계약 취소구분도 못하고좋댄다 ㅋㅋㅋ', 'link': 'https://www.teamblind.com/kr/post/%EC%A7%84%EC%A7%9C-%EA%B1%B0%EB%9E%98%EB%8F%84-%EC%95%88%ED%95%B4%EB%B3%B8-%EB%B6%80%EB%A6%B0%EC%9D%B4%EB%93%A4-%EA%B3%84%EC%95%BD-%EC%B7%A8%EC%86%8C%EA%B5%AC%EB%B6%84%EB%8F%84-%EB%AA%BB%ED%95%98%EA%B3%A0%EC%A2%8B%EB%8C%84%EB%8B%A4-%E3%85%8B%E3%85%8B%E3%85%8B-useu6lxg', 'pre_text': '[Blind] 블라인드에 올라온 글 보셨어요? 광교 거래 취소 속출 중 ㄷㄷㄷ.jpg (부동산) https://www.teamblind.com/kr/s/1z2ttlsx', 'firm': '현대모비스 ', 'id': ' !*********', 'date': '2025-07-02'}
[4번째] {'title': '구성남 vs 하남미사 vs 길음뉴타운 투자 추천', 'link': 'https://www.teamblind.com/kr/post/%EA%B5%AC%EC%84%B1%EB%82%A8-vs-%ED%95%98%EB%82%A8%EB%AF%B8%EC%82%AC-vs-%EA%B8%B8%EC%9D%8C%EB%89%B4%ED%83%80%EC%9A%B4-%ED%88%AC%EC%9E%90-%EC%B6%94%EC%B2%9C-anqy4k0z', 'pre_text': '안녕하세요. 아파트 첫 매수 고민중인 30남 입니다. 원래 대출받아 갭투를 고려하고 있었는데 이번6.27대책으로 대출이 막히는바람에 가진 현금 + 신용대출으로 4억~4.5억 갭투자 고려중입니다. 우선 전 미혼이라 미리 아파트 갭으로 사두고 2년뒤 결혼 후 실거주 계획인데 가치 상승 측면에서 어디가 제일 좋은 선택일지 고수님들 추천 부탁드립니다.

In [21]:
df = pd.DataFrame(list)
# df.to_csv("blind_topic_realestate.csv",encoding="utf-8")
df

,title,link,pre_text,firm,id,date
0,진짜 거래도 안해본 부린이들 계약 취소구분도 못하고좋댄다 ㅋㅋㅋ,https://www.teamblind.com/kr/post/%EC%A7%84%EC...,[Blind] 블라인드에 올라온 글 보셨어요? 광교 거래 취소 속출 중 ㄷㄷㄷ.jp...,현대모비스,!*********,2025-07-02
1,구성남 vs 하남미사 vs 길음뉴타운 투자 추천,https://www.teamblind.com/kr/post/%EA%B5%AC%EC...,안녕하세요. 아파트 첫 매수 고민중인 30남 입니다. 원래 대출받아 갭투를 고려하고...,현대자동차,d********,2025-07-02
2,갭투자 막혔다는게 주담대 때문인가? 세낀 매물 현금으로 사는 건 되는거지?,https://www.teamblind.com/kr/post/%EA%B0%AD%ED...,주담대 안 끼고 세낀 매물 추가 금액(예를 들어 12억 아파트에 6억 전세 끼면 6...,이랜드월드,a********,2025-07-02
3,이고상글에 이거뭐냐,https://www.teamblind.com/kr/post/%EC%9D%B4%EA...,"울이니가 규제해서 두배, 세배 오르고 끝난거랑께!",분당서울대병원,불*****,2025-07-02
4,"6.27 대책 후 서울 계약 취소 164건, 집값 하락 가능성 우려",https://www.teamblind.com/kr/post/627-%EB%8C%8...,https://n.news.naver.com/article/009/000551865...,공무원,i*********,2025-07-02
5,토지 평가 해줄 부동산 전문가 있을까?,https://www.teamblind.com/kr/post/%ED%86%A0%EC...,부동산 컨설팅쪽이나 토지 감정 또는 평가 해줄사람 있을까? 전문가형들 부탁드립니다....,SK실트론,y*****,2025-07-02
6,대출상담사 통한 주담대 중단?,https://www.teamblind.com/kr/post/%EB%8C%80%EC...,우리 NH 이미 중단됐고 국민 내일부터 서류받지말라 했다고 서류같이 준비하던 상담사...,Google Korea,i********,2025-07-02
7,확실히 10-15억대 아파트로 매수가 붙기 시작하네..,https://www.teamblind.com/kr/post/%ED%99%95%EC...,금호역 사거리랑 약수쪽 지난주 금요일 실거래 뜨기 시작하고 이번주 실거래 된것도 많다네.,Google Korea,i********,2025-07-02
8,집 잔금일 이체한도,https://www.teamblind.com/kr/post/%EC%A7%91-%E...,지금 1일 5억이라 올려야 하는데 2주전에 미리 올려둘 수 있어?,서울특별시교육청,l*********,2025-07-02
9,부동산은 고점 찍은거임,https://www.teamblind.com/kr/post/%EB%B6%80%EB...,"왜냐 투자 개같이해서 투자동아리, 투자회사 쫓겨난 내 친구가 최근 갭투했거든",ex-프라이스워터하우스쿠퍼스컨설팅,1****,2025-07-02


In [ ]:
df['article_doc'] = df.apply(lambda x : get_article(x.link) , axis=1)

[세부 글] {'title': '진짜 거래도 안해본 부린이들 계약 취소구분도 못하고좋댄다 ㅋㅋㅋ', 'body': '[Blind] 블라인드에 올라온 글 보셨어요?\n광교 거래 취소 속출 중 ㄷㄷㄷ.jpg (부동산)\nhttps://www.teamblind.com/kr/s/1z2ttlsx', 'firm': '현대모비스', 'id': '· !*********  ', 'like_cnt': '좋아요', 'comment_cnt': '2', 'comment_map': [{'firm': '새회사 ', 'id': ' i*********', 'contents': '\n속출한건 맞음\n'}, {'firm': '새회사 ', 'id': ' !********', 'contents': '\n에휴...\n'}]}
[세부 글] {'title': '구성남 vs 하남미사 vs 길음뉴타운 투자 추천', 'body': '안녕하세요.\n아파트 첫 매수 고민중인 30남 입니다.\n원래 대출받아 갭투를 고려하고 있었는데 이번6.27대책으로 대출이 막히는바람에 가진 현금 + 신용대출으로 4억~4.5억 갭투자 고려중입니다.\n우선 전 미혼이라 미리 아파트 갭으로 사두고 2년뒤 결혼 후 실거주 계획인데 가치 상승 측면에서 어디가 제일 좋은 선택일지 고수님들 추천 부탁드립니다.\n(혹시 그 외 서울 지역 추천해주실곳 있으면 추천 부탁드립니다.)\n구성남 : 산성역 산자푸 20평대\n하남미사: 골든센트로자이 30평대\n길음뉴타운: 래미안8단지 20평대', 'firm': '현대자동차', 'id': '· d********  ', 'like_cnt': '좋아요', 'comment_cnt': '댓글', 'comment_map': []}
[세부 글] {'title': '갭투자 막혔다는게 주담대 때문인가? 세낀 매물 현금으로 사는 건 되는거지?', 'body': '주담대 안 끼고 세낀 매물 추가 금액(예를 들어 12억 아파트에 6억 전세 끼면 6억 추가 내고) 내고 매수하는건 전입의무 없는거지?', 'firm': '이랜

In [156]:
type(df.loc[0]['article_doc']['comment_map'])

list

In [ ]:
# article_doc 을 series 로 반환하는 함수
def extract_article_data(row):
    article_tuple = row['article_doc']
    
    # --- ✨ 가장 중요한 수정 포인트 ---
    # 본격적인 처리를 시작하기 전에 article_tuple이 None인지 먼저 확인합니다.
    # 만약 None이면, 더 이상 진행하지 않고 즉시 None 시리즈를 반환하여 TypeError를 방지합니다.
    if article_tuple is None:
        return pd.Series([None, None, None, None])

    # 이 아래 코드는 article_tuple이 None이 아닐 경우에만 실행됩니다.
    print(f"row : {row}")
    print(f"article_tuple : {article_tuple}")
    
    # 이 부분은 article_doc이 딕셔너리라고 가정하고 작성되었습니다.
    # 만약 문자열이라면 변환 과정이 필요합니다. (아래 '추가 제안' 참고)
    try:
        # 튜플의 첫 번째 요소(제목)와 title 컬럼의 값을 비교
        print(row['title'])
        print(article_tuple['title'])
        
        print(type(row['title']))
        print(type(article_tuple['title']))
        
        if (row['title'] == article_tuple['title']) :
            # 일치하면 body, like_cnt, comment_cnt, comment_map 값을 반환
            res_series = pd.Series([article_tuple['body'], article_tuple['like_cnt'], article_tuple['comment_cnt'], article_tuple['comment_map']])
            
            # print문은 '+'가 아닌 ',' 로 연결해야 합니다.
            print("####", res_series)
            return res_series
        else :
            # 제목이 불일치하면 None으로 채운 Series를 반환
            return pd.Series([None, None, None, None])

    except (TypeError, KeyError) as e:
        # article_tuple이 딕셔너리가 아니거나(예: 튜플, 리스트)
        # 'title' 같은 키가 없을 때 발생할 수 있는 오류를 방지합니다.
        print(f"데이터 처리 중 오류 발생: {e}. 해당 행은 None으로 처리됩니다.")
        return pd.Series([None, None, None, None])

In [162]:
new_cols = ['body', 'like_cnt', 'comment_cnt', 'comment_map']
df[new_cols] = df.apply(extract_article_data, axis=1)

row : title                        진짜 거래도 안해본 부린이들 계약 취소구분도 못하고좋댄다 ㅋㅋㅋ
link           https://www.teamblind.com/kr/post/%EC%A7%84%EC...
pre_text       [Blind] 블라인드에 올라온 글 보셨어요? 광교 거래 취소 속출 중 ㄷㄷㄷ.jp...
firm                                                      현대모비스 
id                                                    !*********
date                                                  2025-07-02
article_doc    {'title': '진짜 거래도 안해본 부린이들 계약 취소구분도 못하고좋댄다 ㅋㅋㅋ...
Name: 0, dtype: object
article_tuple : {'title': '진짜 거래도 안해본 부린이들 계약 취소구분도 못하고좋댄다 ㅋㅋㅋ', 'body': '[Blind] 블라인드에 올라온 글 보셨어요?\n광교 거래 취소 속출 중 ㄷㄷㄷ.jpg (부동산)\nhttps://www.teamblind.com/kr/s/1z2ttlsx', 'firm': '현대모비스', 'id': '· !*********  ', 'like_cnt': '좋아요', 'comment_cnt': '2', 'comment_map': [{'firm': '새회사 ', 'id': ' i*********', 'contents': '\n속출한건 맞음\n'}, {'firm': '새회사 ', 'id': ' !********', 'contents': '\n에휴...\n'}]}
진짜 거래도 안해본 부린이들 계약 취소구분도 못하고좋댄다 ㅋㅋㅋ
진짜 거래도 안해본 부린이들 계약 취소구분도 못하고좋댄다 ㅋㅋㅋ
<class 'str'>
<class 'str'>
#### 0    [Blind] 

In [163]:
df.head() 

,title,link,pre_text,firm,id,date,article_doc,body,like_cnt,comment_cnt,comment_map
0,진짜 거래도 안해본 부린이들 계약 취소구분도 못하고좋댄다 ㅋㅋㅋ,https://www.teamblind.com/kr/post/%EC%A7%84%EC...,[Blind] 블라인드에 올라온 글 보셨어요? 광교 거래 취소 속출 중 ㄷㄷㄷ.jp...,현대모비스,!*********,2025-07-02,{'title': '진짜 거래도 안해본 부린이들 계약 취소구분도 못하고좋댄다 ㅋㅋㅋ...,[Blind] 블라인드에 올라온 글 보셨어요?\n광교 거래 취소 속출 중 ㄷㄷㄷ.j...,좋아요,2,"[{'firm': '새회사 ', 'id': ' i*********', 'conten..."
1,구성남 vs 하남미사 vs 길음뉴타운 투자 추천,https://www.teamblind.com/kr/post/%EA%B5%AC%EC...,안녕하세요. 아파트 첫 매수 고민중인 30남 입니다. 원래 대출받아 갭투를 고려하고...,현대자동차,d********,2025-07-02,"{'title': '구성남 vs 하남미사 vs 길음뉴타운 투자 추천', 'body'...",안녕하세요.\n아파트 첫 매수 고민중인 30남 입니다.\n원래 대출받아 갭투를 고려...,좋아요,댓글,[]
2,갭투자 막혔다는게 주담대 때문인가? 세낀 매물 현금으로 사는 건 되는거지?,https://www.teamblind.com/kr/post/%EA%B0%AD%ED...,주담대 안 끼고 세낀 매물 추가 금액(예를 들어 12억 아파트에 6억 전세 끼면 6...,이랜드월드,a********,2025-07-02,{'title': '갭투자 막혔다는게 주담대 때문인가? 세낀 매물 현금으로 사는 건...,주담대 안 끼고 세낀 매물 추가 금액(예를 들어 12억 아파트에 6억 전세 끼면 6...,좋아요,3,"[{'firm': '새회사 ', 'id': ' 펭***', 'contents': '..."
3,이고상글에 이거뭐냐,https://www.teamblind.com/kr/post/%EC%9D%B4%EA...,"울이니가 규제해서 두배, 세배 오르고 끝난거랑께!",분당서울대병원,불*****,2025-07-02,"{'title': '이고상글에 이거뭐냐', 'body': '울이니가 규제해서 두배,...","울이니가 규제해서 두배, 세배 오르고 끝난거랑께!",좋아요,2,"[{'firm': '새회사 ', 'id': ' !********', 'content..."
4,"6.27 대책 후 서울 계약 취소 164건, 집값 하락 가능성 우려",https://www.teamblind.com/kr/post/627-%EB%8C%8...,https://n.news.naver.com/article/009/000551865...,공무원,i*********,2025-07-02,None,None,None,None,None


In [164]:
df.to_csv('test.csv')